In [ ]:

%cd /content
!git clone https://github.com/lllyasviel/Fooocus.git
%cd /content/Fooocus
!python entry_with_update.py --share --always-high-vram --preset realistic


In [ ]:
import os
import time
import base64 # Import the base64 module
from runwayml import RunwayML

# Configuração do cliente RunwayML
api_key = "key_303185cd2597b2871605f9ebf68eb769ce84313c7c40f25ddf39efc3e8ab0561ba489141b7e6d4e146643b32e11365cd8877d92ebe0c7b80805fb1fa133ec800"  # Substitua pela sua chave de API
client = RunwayML(api_key=api_key)

# Diretórios de entrada e saída
input_dir = "/content/drive/My Drive/bible_code"
output_video = "/content/drive/My Drive/bible_animation.mp4"
os.makedirs(output_dir, exist_ok=True)

# Função para animar uma imagem
def animate_image(image_path, output_path, prompt_text="Animação da cena."):
    with open(image_path, "rb") as img_file:
        image_data = base64.b64encode(img_file.read()).decode('utf-8')

    # Determine image type for data URI prefix
    image_type = os.path.splitext(image_path)[1][1:].lower()  # Get extension and convert to lowercase
    data_uri_prefix = f"data:image/{image_type};base64,"

    # Criação da tarefa de animação, including data URI prefix
    task = client.image_to_video.create(
        model="gen3a_turbo",
        prompt_image=data_uri_prefix + image_data,  # Add the prefix here
        prompt_text=prompt_text,
    )
    task_id = task.id

    # Aguardar a conclusão da tarefa
    while True:
        task_status = client.tasks.retrieve(task_id)
        if task_status.status in ["SUCCEEDED", "FAILED"]:
            break
        time.sleep(10)  # Verifica a cada 10 segundos

    if task_status.status == "SUCCEEDED":
        # Baixar o vídeo gerado
        video_url = task_status.output.video
        video_data = client._client.get(video_url).content
        with open(output_path, "wb") as video_file:
            video_file.write(video_data)
        print(f"Animação salva em: {output_path}")
    else:
        print(f"Falha ao animar {image_path}")

# Processar todas as imagens na pasta de entrada
for i in range(1, 7):
    img_filename = f"scene_{i}.png"
    img_path = os.path.join(input_dir, img_filename)
    output_path = os.path.join(output_dir, f"scene_{i}_animated.mp4")

    if os.path.exists(img_path):
        animate_image(img_path, output_path)
    else:
        print(f"Imagem não encontrada: {img_path}")

print("Processo de animação concluído.")
